# AI Community @ Семинар № 7, весна 2018
## State-of-the-art в компьютерном зрении. SqueezeNet, MobileNet, DenseNet

## SqueezeNet [2017, ICLR]

[SQUEEZENET: ALEXNET-LEVEL ACCURACY WITH 50X FEWER PARAMETERS AND <0.5MB MODEL SIZE](https://arxiv.org/abs/1602.07360)

### Ключевые особенности

SqueezeNet предлагает очень сжатую сеть с сохранением высокой точности классификации. Это дает:  
1. Быстрее обучение
1. Модели занимаеют меньше памяти -> легче передавать клиенту
1. Возможность запускать на устройствах с малым объемом памяти (Raspberry Pi, FPGA, etc.)

### Дизайн

Для сжатия сети использованы следующие стратегии:
1. Заменить свертки `3x3` на `1x1`
1. Уменьшить кол-во входных каналов для оставшихся сверток `3x3`
1. Уменьшать пространство в поздних слоях (для сохранения высокой точности)

#### Fire module

![SqueezeNet fire module](images/squeeze_fire_module.png)

Состоит из одного `1x1` squeeze слоя (для уменьшения пространства карт), и двух expand слоев - один является сверткой `1x1`, а другой сверткой `3x3`. Squeeze слой ответственен за стратегию 2, уменьшая размерность входных карт для expand слоя.
Для осуществления этого кол-во карт в слое squeeze $s_{1x1}$ должно быть меньше суммарного кол-ва карт в expand слоях ($e_{1x1}+e_{3x3}$)

#### Финальная архитектура

![Архитектура SqueezeNet](images/squeeze_architecture.png)

### Результаты

При определенных параметрах $s_{1x1}, e_{1x1}, e_{3x3}$ сеть достигает сжатия в 50x по сравнению с базовой архитектурой AlexNet. При этом точность top-1 и top-5 остаются прежними или даже лучше в некоторых случаях, чем у AlexNet.

Помимо этого, после применения техники [Deep Compression](https://arxiv.org/abs/1510.00149) сеть можно сжать еще в 10 раз, сохранив при этом точность классификации.  
Финальная модель авторов содержит **510x** меньше весов, чем базовый AlexNet. Модель при этом имеет размер **0.47MB**, что очень удобно для поставки клиентам (например, в мобильное приложение).

Также в сети становится намного меньше параметров. **~0.4 млн** против ~60 млн (в 50x меньше после применения дизайна SqueezeNet и еще 3x меньше после Deep Compression).

Top-1 accuracy on ImageNet: **60.4%**  
Top-5 accuracy on ImageNet: **82.5%**

## MobileNet

[MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications](https://arxiv.org/abs/1704.04861)

### Ключевые особенности

1. Эффективная архитектура нейронной сети
1. Набор из двух гиперпараметров, которые позволяют построить сеть под конкретные условия (задача, порог точности, железо)

### Дизайн

#### Depthwise separable convolution

![Depthwise separable convolution](images/mobile_depthwise_conv.png)

Стоимость обычной свертки: $D_K \cdot D_K \cdot M \cdot N \cdot D_F \cdot D_F$  
Стоимость depthwise separable convolution: $D_K \cdot D_K \cdot M \cdot D_F \cdot D_F + M \cdot N \cdot D_F \cdot D_F$  
Уменьшение стоимости: $\frac{1}{N} + \frac{1}{D^2_K}$

#### Архитектура

![Архитектура MobileNet](images/mobile_architecture.png)

#### Width Multiplier

Мы можем уменьшать стоимость вычислений, добавив параметр сжатия сверток $\alpha \in (0, 1]$. Тогда сложность вычислений depthwise separable convolution будет $D_K \cdot D_K \cdot \alpha M \cdot D_F \cdot D_F + \alpha M \cdot \alpha N \cdot D_F \cdot D_F$  
При этом сложность уменьшается примерно в $\alpha^2$

#### Resolution Multiplier

Также мы можем уменьшать размерность входных карт. Для этого используется параметр $\rho$. Результирующая сложность представлена $D_K \cdot D_K \cdot \alpha M \cdot \rho D_F \cdot \rho D_F + \alpha M \cdot \alpha N \cdot \rho D_F \cdot \rho D_F$  
Таким образом вычислительная сложность еще падает в $\rho^2$ раз

Top-1 accuracy on ImageNet: **70.6%**